In [1]:
import json
graph_nodes = json.loads(open('tst.json', 'r').read())['data']
graph_nodes

[{'id': '9b0eff41-1ff2-4f8f-937c-ffc436ed9c4e',
  'nodeType': 'input',
  'data': {'text': 'Hilary'},
  'pointingTo': ['290ab4f2-4624-4894-b337-91b12fd60dc3']},
 {'id': 'b2da3bc2-eabb-4869-9af9-b38c86e01f69',
  'nodeType': 'input',
  'data': {'text': 'Jordan'},
  'pointingTo': ['290ab4f2-4624-4894-b337-91b12fd60dc3']},
 {'id': '290ab4f2-4624-4894-b337-91b12fd60dc3',
  'nodeType': 'generate',
  'data': {'text': 'convert {} and {} into one json'},
  'pointedBy': ['9b0eff41-1ff2-4f8f-937c-ffc436ed9c4e',
   'b2da3bc2-eabb-4869-9af9-b38c86e01f69'],
  'pointingTo': ['85e3cf82-aa1f-4c35-a67d-7fc13164f9df']},
 {'id': '51308df1-7782-4cbf-a8db-980195a3c192',
  'nodeType': 'output',
  'data': {'text': ''},
  'pointedBy': ['85e3cf82-aa1f-4c35-a67d-7fc13164f9df']},
 {'id': '85e3cf82-aa1f-4c35-a67d-7fc13164f9df',
  'nodeType': 'generate',
  'data': {'text': 'review and add other data to {}'},
  'pointedBy': ['290ab4f2-4624-4894-b337-91b12fd60dc3'],
  'pointingTo': ['51308df1-7782-4cbf-a8db-980195a3c1

In [2]:
# map_of_nodes - holds all nodes with all data from graph source of all information
# nodes_to_execute - holds nodes "id" that are curently loadaed to be processed or wait until resolve earlier and then starts preocessing 
# map_of_processed_nodes - holds only output of llm

# for start of resolving loop
nodes_to_execute = [x['id'] for x in graph_nodes if 'pointedBy' not in x.keys()]
executed_nodes_in_iteration = []
# add all input nodes
executed_nodes_in_iteration.extend(nodes_to_execute)

# for easy access by pointing id
map_of_nodes = {}
for x in graph_nodes:
    map_of_nodes[x['id']] = x

map_of_processed_nodes = {}
# pass input nodes data straight to context
for x in nodes_to_execute:
    map_of_processed_nodes[x] = map_of_nodes[x]

import logging
logger = logging.getLogger(__name__)

In [3]:
number_of_loops = 0    
while len(map_of_processed_nodes.keys()) != len(map_of_nodes.keys()):
    for node_id in nodes_to_execute:
        print(node_id)
        keys = map_of_processed_nodes.keys()
        if map_of_nodes[node_id]['nodeType'] == "generate" and  all(key in keys  for key in map_of_nodes[node_id]['pointedBy']):
            # ducktape prompt context
            prompt = map_of_nodes[node_id]['data']['text']
            context = [node for node in map_of_nodes[node_id]['pointedBy']]
            context = [map_of_processed_nodes[idx] for idx in context]
            prompt = prompt.format(*context)
            print(prompt)
            map_of_processed_nodes[node_id] = prompt
        else:        
            map_of_processed_nodes[node_id] = map_of_nodes[node_id]['data']['text']
        executed_nodes_in_iteration.append(node_id)
        if (map_of_nodes[node_id]['nodeType'] != 'output'):
            nodes_to_execute.extend(map_of_nodes[node_id]['pointingTo'])
        # logger.info("Processing node with id:", node_id)
        nodes_to_execute = list(set(nodes_to_execute) - set(executed_nodes_in_iteration))
        executed_nodes_in_iteration = []
   

9b0eff41-1ff2-4f8f-937c-ffc436ed9c4e
b2da3bc2-eabb-4869-9af9-b38c86e01f69
290ab4f2-4624-4894-b337-91b12fd60dc3
convert Hilary and Jordan into one json
85e3cf82-aa1f-4c35-a67d-7fc13164f9df
review and add other data to convert Hilary and Jordan into one json
51308df1-7782-4cbf-a8db-980195a3c192
